
 <div style="text-align: right; font-size: 10px; color:purple"> Curso de Especialización en Inteligencia Artificial y Big Data <br>
 Programación de Inteligencia Artificial </div>


---



# Ejercicio 3: Correspondencias de puntos (Óscar López Arcos)

In [ ]:
!pip install opencv-python

In [ ]:
import cv2
import numpy as np
import gdown
from matplotlib import pyplot as plt
import random as rdm

En esta actividad vamos a hacer uso de diferentes herramientas que proporciona OpenCV para realizar operaciones y cálculos sobre imágenes.

Primero de todo, vamos a añadir a nuestro entorno las imágenes necesarias para el desarrollo del ejercicio. Para ello usaremos la librería `gdown`, con el ID de la carpeta pública de Google Drive.

In [ ]:
gdown.download_folder(id='1InzY0PJ8QeRE4m_BuOE1pBVfLuGoclN7')

A continuación, vamos a definir una serie de constantes que usaremos a lo largo de los diferentes ejercicios, y definir la ruta donde vais a guardar vuestras imágenes.

In [ ]:
def read_img( filename ):
    img = cv2.imread( filename )
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

def show_img( imagen, nombre ):
    plt.imshow(imagen)
    plt.title(nombre)
    plt.axis('off')
    plt.show()

# Constantes
sift = cv2.xfeatures2d.SIFT_create(contrastThreshold = 0.06)
rdm.seed(1)

# Modificar segun vuestra ruta
ruta_imagenes = '/content/imagenes/'

En el siguiente fragmento de código, definimos una función que, dadas dos imágenes:
- En primer lugar, calcula los puntos clave de cada imagen (key points). Estos puntos son aquellos suficientemente distintivos, es decir, que representan una característica única y estable en una imagen.
- A continuación, calculamos la correspondencia de puntos clave. Esto nos devuelve una estructura (DMatch) que contiene los k mejores puntos coincidentes para cada rasgo de la Imagen 1 dentro de la Imagen 2.
- Por último, dibuja las imágenes con la correspondencia de puntos.

In [ ]:
def drawMatches(imagen1, imagen2, improve=True):

    # Calculamos los descriptores
    kp1, des1 = sift.detectAndCompute(imagen1,None)
    kp2, des2 = sift.detectAndCompute(imagen2,None)

    # Emparejamos descriptores
    matches = cv2.BFMatcher().knnMatch(des1, des2, k = 2)

    # Aplicar ratio de 0.7 (Lowe)
    if improve:
        good_matches = []
        for m,n in matches:
            if m.distance < 0.7*n.distance:
                good_matches.append([m])
    else:
        good_matches = np.array(matches)

    # img4 = cv2.drawMatchesKnn(imagen1, kp1, imagen2, kp2, good_matches[:100],outImg=None,flags=2)
    img4 = myCustomDrawMatchesKnn(imagen1,kp1,imagen2,kp2,good_matches[:100])
    show_img(img4, 'KnnMatch')

img1 = read_img(ruta_imagenes+ 'alhambra1.jpg')
img2 = read_img(ruta_imagenes+ 'alhambra2.jpg')

drawMatches(img1, img2)

En el fragmento de código anterior, justo después de llamar a `drawMatchesKnn`, hay una línea comentada 😏. La siguiente tarea consiste en implementar esa función (que debe cumplir el mismo propósito) y sustituir la llamada. La siguiente plantilla hará que sea más sencillo.

In [ ]:
# @title
def myCustomDrawMatchesKnn(i1, kp1, i2, kp2, matches, thickness=1):
    """
    Dibuja coincidencias entre dos imágenes manualmente, reemplazando cv2.drawMatchesKnn.

    Args:
        i1 (np.ndarray): Primera imagen.
        kp1 (tuple): Puntos clave de la primera imagen.
        i2 (np.ndarray): Segunda imagen.
        kp2 (tuple): Puntos clave de la segunda imagen.
        matches (list): Lista de coincidencias (DMatch).
        thickness (int): Grosor de las líneas.

    Returns:
        np.ndarray: La imagen combinada con las líneas dibujadas.
    """

    img_out = None

    # --- 1. Dibujar ambas imágenes ---

    # --- 1. 1. Preparar un marco (imagen negra) donde quepan ambas imágenes.
    alto1, ancho1, canal1 = i1.shape
    alto2, ancho2, canal2 = i2.shape

    alto = max(alto1, alto2)
    ancho = ancho1 + ancho2
    frame = np.zeros((alto,ancho,3), dtype=np.uint8)

    # --- 1. 2. Pegar ambas imágenes en el marco, una al lado de otra

    frame[0:alto1, 0:ancho1, 0:canal1] = i1
    frame[0:alto2, ancho1:, 0:canal2] = i2

    # --- 2. Dibujar las Líneas ---

    for match in matches:
        # Los matches filtrados por Lowe ya son una lista de listas [[DMatch]],
        # por lo que tomamos el primer (y único) objeto DMatch
      m = match[0]
        # --- 2.1. Calcular ambos puntos en la imagen
        # https://docs.opencv.org/4.x/d4/de0/classcv_1_1DMatch.html
      puntosI1 = kp1[m.queryIdx].pt
      puntosI2 = kp2[m.trainIdx].pt
      xI1 = int(puntosI1[0])
      yI1 = int(puntosI1[1])
      xI2 = int(puntosI2[0])
      yI2 = int(puntosI2[1])
        # --- 2.2. Dibujar ambos puntos en la imagen, unidos por una línea de
        # de color aleatorio
        # https://docs.opencv.org/4.x/dc/da5/tutorial_py_drawing_functions.html
      img_out = cv2.line(frame, (xI1, yI1) , (xI2 + ancho1, yI2)  , (rdm.randint(0, 255),rdm.randint(0, 255),rdm.randint(0, 255)) ,thickness)

    return img_out

Pero, ¿para qué sirve todo esto? A continuación, veremos como calcular las equivalencias de puntos clave entre varias imágenes para posteriormente aplicar una serie de operaciones. De esta forma podremos construir mosaicos de forma parecida a como vuestro teléfono móvil toma fotos panorámicas.

In [ ]:
# Mueve la imagen img al centro del marco framde
def move_to_center(img, frame):
    # Se obtienen las dimensiones de la imagen central
    height_frame, width_frame, B = frame.shape
    height_img, width_img, A = img.shape

    # Se calculan las coordenadas de inicio de la imagen central
    y_displacement = height_frame/2 - height_img/2
    x_displacement = width_frame/2 - width_img/2

    # Se crea una matriz 3x3 con los valores a 0 menos la diagonal a 1 y los
    # valores de la traslación
    m0 = np.array([[1, 0, x_displacement], [0, 1, y_displacement], [0, 0, 1]], dtype=np.float32)

    # BORDER_CONSTANT para rellenar de negro
    frame = cv2.warpPerspective(img, m0, (width_frame, height_frame), dst=frame, borderMode=cv2.BORDER_CONSTANT)

    return frame, m0

# Elimina los márgenes negros de una imagen
def crop_image(img):
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Nos quedamos con los colores distintos de negro
    _, thresh = cv2.threshold(gray, 1, 255, cv2.THRESH_BINARY)
    # Obtenemos el contorno
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # Se calculan las esquinas
    x, y, w, h = cv2.boundingRect(np.concatenate(contours))
    # Limitamos la imagen
    return img[y:y+h,x:x+w]

def knnMatch(i1, i2, improve=True):
    # Creamos el objeto de correspondencias
    bf = cv2.BFMatcher()

    # Calculamos los descriptores
    kp1, des1 = sift.detectAndCompute(i1,None)
    kp2, des2 = sift.detectAndCompute(i2,None)

    # Emparejamos descriptores
    matches = bf.knnMatch(des1, des2, k = 2)

    # Aplicar ratio de 0.7 (Lowe)
    if improve:
        good = []
        for m,n in matches:
            if m.distance < 0.7*n.distance:
                good.append([m])
    else:
        good = np.array(matches)

    return ((kp1,des1),(kp2,des2), good)

def get_homeography(i1,i2):

    # Se calculan las correspondencias
    kpdes1, kpdes2, matches = knnMatch(i1,i2,improve=True)

    kp1, des1 = kpdes1
    kp2, des2 = kpdes2

    # Comprobamos tener el mínimo para realizar homografía
    if len(matches)<4:
        return -1

    # Cambiamos el formato de los puntos
    src_pts = np.float32([ kp1[m[0].queryIdx].pt for m in matches ]).reshape(-1,1,2)
    dst_pts = np.float32([ kp2[m[0].trainIdx].pt for m in matches ]).reshape(-1,1,2)

    # Calculamos homografía
    M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 1)

    return M

# Compone el mosaico paso a paso, pegando una imagen detrás de otra
def mosaico_R(imagenes):

    # Se calcula el ancho del mosaico como la suma de todos los anchos de las imágenes
    height = sum([i.shape[0] for i in imagenes])
    width = sum([i.shape[1] for i in imagenes])

    # construimos lienzo o marco
    frame = np.zeros((height,width,3), dtype=np.uint8)

    # Movemos primera imagen al centro
    img, m0 = move_to_center(imagenes[0],frame)

    # Calculamos homografías y llevamos imágenes al mosaico una a una
    for i in range(1, len(imagenes)):
        M = get_homeography(imagenes[i],img)
        img = cv2.warpPerspective(imagenes[i], M, (width, height), dst=img, borderMode=cv2.BORDER_TRANSPARENT)

    # Recortamos los bordes
    img = crop_image(img)
    return img

# Calcula previamente todas las homografías necesarias para trasladar cada imagen
# al mosaico y por último las traslada
def mosaico_M(imagenes):
    # Se calcula el ancho del mosaico como la suma de todos los anchos de las imágenes
    height = sum([i.shape[0] for i in imagenes])
    width = sum([i.shape[1] for i in imagenes])

    # construimos lienzo o marco
    frame = np.zeros((height,width,3), dtype=np.uint8)

    # Obtenemos la homografía necesaria para llevar la imagen al centro del marco
    img, m0 = move_to_center(imagenes[0],frame)

    homografias = []
    homografias.append(m0)

    # Calculamos todas las homeografías necesarias para formar el mosaico
    for i in range(1, len(imagenes)):
        # Homografía de imágenes dos a dos
        M = get_homeography(imagenes[i], imagenes[i-1])
        # Multiplicamos por la homografía del paso anterior
        new_M = np.matmul(m0, M)
        homografias.append(new_M)
        m0 = new_M

    # Trasladamos todas las imágenes al marco
    for i in range(len(imagenes)):
        img = cv2.warpPerspective(imagenes[i], homografias[i], (width, height), dst=img, borderMode=cv2.BORDER_TRANSPARENT)

    # Recortamos los bordes
    img = crop_image(img)
    return img

def calcularMosaicos( imagenes ):
    im = mosaico_R(imagenes)
    show_img(im, 'mosaico_R')
    im = mosaico_M(imagenes)
    show_img(im, 'mosaico_M')

img1 = read_img(ruta_imagenes+'alhambra1.jpg')
img2 = read_img(ruta_imagenes+'alhambra2.jpg')
img3 = read_img(ruta_imagenes+'alhambra3.jpg')

calcularMosaicos([img1,img2,img3])